In [ ]:
# !pip install --upgrade transformers
# !pip install simpletransformers
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Gen RAM Free: 26.4 GB  |     Proc size: 110.4 MB
GPU RAM Free: 16130MB | Used: 0MB | Util   0% | Total     16130MB


In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False


In [ ]:
train_df = pd.read_csv(r'/content/train.csv',sep=";",encoding='utf-8')
test_df  = pd.read_csv(r'/content/test_reduced.csv',sep=";",encoding='utf-8')
train_df['Text'] = train_df['MailSubject']+' '+train_df['MailTextBody']
test_df['Text']  = test_df['MailSubject']+' '+test_df['MailTextBody']

train_df=train_df.dropna(subset=['ManualGroups'])

In [ ]:
train_df['Text']=train_df['Text'].str.replace('\\r',' ')
train_df['Text']=train_df['Text'].str.replace('\\n','')
train_df['Text']=train_df['Text'].str.replace('\r','')
train_df['Text']=train_df['Text'].str.replace('\n','')
train_df['Text']=train_df['Text'].str.replace('  ','')
train_df['Text']=train_df['Text'].str.lower()
train_df['Text']=train_df['Text'].str.replace('&nbsp;','')
train_df['Text']=train_df['Text'].str.strip()


test_df['Text']=test_df['Text'].str.replace('\\r',' ')
test_df['Text']=test_df['Text'].str.replace('\\n','')
test_df['Text']=test_df['Text'].str.replace('\r','')
test_df['Text']=test_df['Text'].str.replace('\n','')
test_df['Text']=test_df['Text'].str.replace('  ','')
test_df['Text']=test_df['Text'].str.lower()
test_df['Text']=test_df['Text'].str.replace('&nbsp;','')
test_df['Text']=test_df['Text'].str.strip()

In [ ]:
class_list=train_df.ManualGroups.unique().tolist()
train_df['pred_class']=train_df.apply(lambda x:  class_list.index(x['ManualGroups']),axis=1)

In [ ]:
train_df1=train_df[['Text','pred_class']]
train_df1['Text']=train_df1['Text'].astype(str)
train_df1.head()

,Text,pred_class
3,"botschaft warschau -administratoren-zugang für ms teamsmd5:95c1b3200cb6e564b7af577b457cbc23 sehr geehrte damen und herren,die botschaft in warschau plant am 18. november ein webinar zu organisiere...",0
8,webinare für aussenstehende: welche plattformen eignen sich? md5:ee25fc7c9144a9186c79468131712b9a sehr geehrte damen und herrenwir werden in 3 wochen auf der schweizerischen botschaft in polen ein...,0
15,"failure notice md5:cbc4aeb1e66edb33e018bdbab24e871e liebe kolleg/innen,es ist ein bisschen beunruhigend, wenn eine email meiner tochter https://www.admin.ch> nicht auf mein outlook übermittelt wir...",1
26,"kein zugriff auf intranet eda md5:0a754b13d3b4f21c55defba1aff914ea guten tagich habe das gleiche problem wie gestern morgen: kein zugriff auf intranet eda. heute dreht es im leeren, gestern kam ei...",2
29,ich komme nicht ins intranet / internet md5:c75ecf1bfe1805d21e156283ad9509dd guten morgentrotz neustart komme ich heute morgen nicht ins intranet/internet. folgendes bild erscheint. herzlichen dan...,2


In [ ]:
from sklearn.model_selection import train_test_split

train_df_, test_df_ = train_test_split(train_df1, test_size=0.05)

print('train shape: ',train_df_.shape)
print('test shape: ',test_df_.shape)

train shape:  (864, 2)
test shape:  (46, 2)


In [ ]:
model = ClassificationModel("xlmroberta", "xlm-roberta-large-finetuned-conll03-german", use_cuda=True,num_labels=69, 
                                                                  args={'train_batch_size':2,
                                                                        'reprocess_input_data': True,
                                                                        'overwrite_output_dir': True,
                                                                        'fp16': False,
                                                                        'do_lower_case': True,
                                                                        'num_train_epochs': 9,
                                                                        'max_seq_length': 512,
                                                                        'regression': False,
                                                                        'manual_seed': 2,
                                                                        "learning_rate":3e-5,
                                                                        'weight_decay':0,
                                                                        "save_eval_checkpoints": True,
                                                                        "save_model_every_epoch": True,
                                                                        "silent": False})
model.train_model(train_df_)

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-german were not used when initializing XLMRobertaForSequenceClassification: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large-finetuned-conll03-german and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df_, f1=f1_multiclass, acc=accuracy_score)

result

In [ ]:
preds=model.predict(test_df['Text'])[0]
len(preds)

In [ ]:
output_df=pd.DataFrame(test_df['Id'])
output_df['Predicted_labels']=preds
output_df.head()

In [ ]:
output_df['Predicted']=output_df['Predicted_labels'].apply(lambda x:  class_list[x])
output_df['Predicted'].value_counts()/output_df.shape[0]

In [ ]:
output_df[['Id','Predicted']].to_csv('sample_trial_german_bert_manual_16_512.csv',index=False)

In [ ]:
output_df